In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

# NB: add more subjects in the data folder in neurolib package
from neurolib.utils.loadData import Dataset
from neurolib.models.aln import ALNModel

from coonectivity_dynamics import (plot_kuramoto_example, kuramoto,
                                   fast_kuramoto)
plt.style.use('seaborn')
sns.set_style("whitegrid")
sns.set_context("talk")

In [ ]:
ds = Dataset("gw")
model = ALNModel(Cmat=ds.Cmats[26], Dmat=ds.Dmats[26])
model.params['dt'] = 0.1
model.params['duration'] = 20 * 1000  # ms

# add custom parameter for downsampling results
# 10 ms sampling steps for saving data, should be multiple of dt
model.params['save_dt'] = 10.0
model.params["tauA"] = 600.0
model.params["sigma_ou"] = 0.0
model.params["b"] = 20.0

model.params["Ke_gl"] = 300.0
model.params["signalV"] = 80.0
model.params["mue_ext_mean"] = 1.5
model.params["mui_ext_mean"] = 0.2


In [ ]:
model.run()

In [ ]:
model.default_output

In [ ]:
plt.figure(figsize=(12, 6))
plt.imshow(model.rates_exc[:, :10000], aspect='auto') # Aspect=1.2

In [ ]:
plot_kuramoto_example(model.rates_exc[:4, 5000:15_000])

### Check the time profile of the kuramoto estimation function

In [ ]:
from line_profiler import LineProfiler
lp = LineProfiler()
lp_wrapper = lp(kuramoto)
kur = lp_wrapper(model.rates_exc[::10])
lp.print_stats()

In [ ]:
%%time
kur1 = fast_kuramoto(model.rates_exc[::10][:, :10_000])


In [ ]:
%%time
kur2 = kuramoto(model.rates_exc[::10][:, :10_000])

In [ ]:
np.corrcoef(kur1, kur2)